# production code

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType
from pyspark.sql.functions import col
from pyspark.sql.types import StringType, StructType, StructField


In [2]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder
                .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.1,org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.7.1,org.apache.hadoop:hadoop-aws:3.3.2")
                .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000") 
                .config("spark.hadoop.fs.s3a.access.key", "minio") 
                .config("spark.hadoop.fs.s3a.secret.key", "minio123") 
                .config("spark.hadoop.fs.s3a.path.style.access", "true") 
                .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
                .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog")
                .config("spark.sql.catalog.spark_catalog.type", "hadoop")
                .config("spark.sql.catalog.spark_catalog.warehouse", "s3a://commerce/warehouse/") 
                .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") 
                .appName("Kafka to MinIO Pipeline")
         .getOrCreate())

25/02/10 22:13:04 WARN Utils: Your hostname, Gauravs-MacBook-Air-3.local resolves to a loopback address: 127.0.0.1; using 192.168.1.10 instead (on interface en0)
25/02/10 22:13:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/Users/saurabh/Library/Caches/pypoetry/virtualenvs/realtime-fraud-detection-system-eUiKKk2v-py3.13/lib/python3.13/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/saurabh/.ivy2/cache
The jars for the packages stored in: /Users/saurabh/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ba0a1697-3bdc-474f-b48a-19ff0897d01d;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.1 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found commons-logging#commons-logging;1.1.3 in ce

In [60]:
def execute_query(query: str):
    spark.sql(query).show(truncate=False)

In [61]:
# Kafka configurations
kafka_brokers = "localhost:9093"  # Update with your Kafka broker address
kafka_topic = "debezium.payment.customers"
# Read data from Kafka (batch mode)
kafka_df = spark.read \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_brokers) \
    .option("subscribe", kafka_topic) \
    .option("startingOffsets", "earliest") \
    .load()

In [62]:
kafka_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [63]:
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType, StringType, LongType

# Define the schema for the JSON messages
column_schema = StructType([
StructField("schema", StringType(), False),
StructField("payload", StructType([
    StructField("before", StructType([
        StructField("customer_id", IntegerType(), False),
        StructField("x_location", DoubleType(), False),
        StructField("y_location", DoubleType(), False)
    ]), True),
    StructField("after", StructType([
        StructField("customer_id", IntegerType(), False),
        StructField("x_location", DoubleType(), False),
        StructField("y_location", DoubleType(), False)
    ]), True),
    StructField("source", StructType([
        StructField("version", StringType(), False),
        StructField("connector", StringType(), False),
        StructField("name", StringType(), False),
        StructField("ts_ms", LongType(), False),
        StructField("snapshot", StringType(), True),
        StructField("db", StringType(), False),
        StructField("sequence", StringType(), True),
        StructField("ts_us", LongType(), True),
        StructField("ts_ns", LongType(), True),
        StructField("schema", StringType(), False),
        StructField("table", StringType(), False),
        StructField("txId", LongType(), True),
        StructField("lsn", LongType(), True),
        StructField("xmin", LongType(), True)
    ]), False),
    StructField("transaction", StructType([
        StructField("id", StringType(), False),
        StructField("total_order", LongType(), False),
        StructField("data_collection_order", LongType(), False)
    ]), True),
    StructField("op", StringType(), False),
    StructField("ts_ms", LongType(), True),
    StructField("ts_us", LongType(), True),
    StructField("ts_ns", LongType(), True)
]))])

from pyspark.sql.functions import from_json

kafka_df = kafka_df.withColumn("value", col("value").cast(StringType())) \
    .withColumn("key", col("key").cast(StringType()))
# Apply the schema to the DataFrame
kafka_df = kafka_df.withColumn("value", from_json("value", column_schema))
kafka_df.createOrReplaceTempView("kafkadata")

In [65]:
execute_query("select * from kafkadata")

25/02/10 23:32:14 WARN AdminClientConfig: The configuration 'key.deserializer' was supplied but isn't a known config.
25/02/10 23:32:14 WARN AdminClientConfig: The configuration 'value.deserializer' was supplied but isn't a known config.
25/02/10 23:32:14 WARN AdminClientConfig: The configuration 'enable.auto.commit' was supplied but isn't a known config.
25/02/10 23:32:14 WARN AdminClientConfig: The configuration 'max.poll.records' was supplied but isn't a known config.
25/02/10 23:32:14 WARN AdminClientConfig: The configuration 'auto.offset.reset' was supplied but isn't a known config.


+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [77]:
execute_query(""" create or replace temp view latest_data as 
              with extracted_data  as (
                    select value.payload.after.customer_id,
                    value.payload.after.x_location, 
                    value.payload.after.y_location,
                    timestamp
                    from kafkadata),
              ranked_data as (
                  select customer_id,
                  x_location,
                  y_location, 
                  timestamp,
                  ROW_NUMBER() over (partition by customer_id order by timestamp DESC) as rn
                  from extracted_data) 
              select customer_id,
                  x_location,
                  y_location,
                  now() as row_created_timestamp,
                  now() as row_updated_timestamp
              from ranked_data 
              where rn=1""")

++
||
++
++



In [70]:
execute_query("CREATE DATABASE IF NOT EXISTS payment;")
execute_query("show databases")

++
||
++
++

+---------+
|namespace|
+---------+
|default  |
|payment  |
+---------+



In [81]:
execute_query("""
        CREATE TABLE IF NOT EXISTS payment.customer (
      customer_id INT,
      x_location FLOAT,
      y_location FLOAT,
      row_created_timestamp TIMESTAMP,
      row_updated_timestamp TIMESTAMP
)
USING iceberg
""")

++
||
++
++



In [82]:
execute_query("""
MERGE INTO payment.customer AS target
USING latest_data AS source
ON target.customer_id = source.customer_id
WHEN MATCHED THEN
  UPDATE SET
    target.x_location = source.x_location,
    target.y_location = source.y_location,
    target.row_updated_timestamp = source.row_updated_timestamp
WHEN NOT MATCHED THEN
  INSERT (
    customer_id,
    x_location,
    y_location,
    row_created_timestamp,
    row_updated_timestamp
  )
  VALUES (
    source.customer_id,
    source.x_location,
    source.y_location,
    source.row_created_timestamp,
    source.row_updated_timestamp
  )
""")

25/02/11 00:23:07 WARN AdminClientConfig: The configuration 'key.deserializer' was supplied but isn't a known config.
25/02/11 00:23:07 WARN AdminClientConfig: The configuration 'value.deserializer' was supplied but isn't a known config.
25/02/11 00:23:07 WARN AdminClientConfig: The configuration 'enable.auto.commit' was supplied but isn't a known config.
25/02/11 00:23:07 WARN AdminClientConfig: The configuration 'max.poll.records' was supplied but isn't a known config.
25/02/11 00:23:07 WARN AdminClientConfig: The configuration 'auto.offset.reset' was supplied but isn't a known config.
25/02/11 00:23:08 WARN AdminClientConfig: The configuration 'key.deserializer' was supplied but isn't a known config.
25/02/11 00:23:08 WARN AdminClientConfig: The configuration 'value.deserializer' was supplied but isn't a known config.
25/02/11 00:23:08 WARN AdminClientConfig: The configuration 'enable.auto.commit' was supplied but isn't a known config.
25/02/11 00:23:08 WARN AdminClientConfig: The c

++
||
++
++



In [83]:
execute_query("""select * from payment.customer  """)

+-----------+----------+----------+--------------------------+--------------------------+
|customer_id|x_location|y_location|row_created_timestamp     |row_updated_timestamp     |
+-----------+----------+----------+--------------------------+--------------------------+
|0          |54.88135  |71.518936 |2025-02-11 00:23:07.265899|2025-02-11 00:23:07.265899|
|1          |60.276337 |54.48832  |2025-02-11 00:23:07.265899|2025-02-11 00:23:07.265899|
|2          |50.4567   |64.58941  |2025-02-11 00:23:07.265899|2025-02-11 00:23:07.265899|
|3          |64.58941  |43.75872  |2025-02-11 00:23:07.265899|2025-02-11 00:23:07.265899|
|4          |43.75872  |89.1773   |2025-02-11 00:23:07.265899|2025-02-11 00:23:07.265899|
|5          |89.1773   |96.36628  |2025-02-11 00:23:07.265899|2025-02-11 00:23:07.265899|
|6          |96.36628  |38.34415  |2025-02-11 00:23:07.265899|2025-02-11 00:23:07.265899|
|7          |38.34415  |79.1725   |2025-02-11 00:23:07.265899|2025-02-11 00:23:07.265899|
|8        

In [75]:
execute_query("""select * from payment.customers """)

+-----------+----------+----------+
|customer_id|x_location|y_location|
+-----------+----------+----------+
|0          |54.88135  |71.518937 |
|1          |60.276338 |54.488318 |
|2          |50.4567   |64.589411 |
|3          |64.589411 |43.758721 |
|4          |43.758721 |89.1773   |
|5          |89.1773   |96.366276 |
|6          |96.366276 |38.344152 |
|7          |38.344152 |79.172504 |
|8          |79.172504 |52.889492 |
|9          |52.889492 |56.804456 |
|10         |40.7128   |74.006    |
+-----------+----------+----------+



In [ ]:
postgres -> debezium -> kafka (retention policy - 7 days ) -> spark streaming -> minio (s3)

In [ ]:
kafka -> topic -> transform -> s3 table (create / replace )

incremental processing 

1. kafka -> event -> 

incremental read kafka basis offset -> spark streaming -> checkpointing (state store )

triggers -> freuently job execution





done this:
10 row -> updates + inserts + delete  upsert -> key column 

merge command (iceberg)


table - 100 rows 

8 am 

